In [15]:
import pandas as pd

In [16]:
#
#
#
#
#
def dictSetup():
    global team
    global mpd
    global ast
    global ctl
    global hyb
    global esc
    
    team = dict()
    team["Atlanta Reign"] = "ATL"
    team["Boston Uprising"] = "BOS"
    team["Chengdu Hunters"] = "CHD"
    team["Dallas Fuel"] = "DAL"
    team["Florida Mayhem"] = "FLO"
    team["Guangzhou Charge"] = "GZC"
    team["Hangzhou Spark"] = "HZS"
    team["Houston Outlaws"] = "HOU"
    team["London Spitfire"] = "LDN"
    team["Los Angeles Gladiators"] = "GLA"
    team["Los Angeles Valiant"] = "VAL"
    team["New York Excelsior"] = "NYE"
    team["Paris Eternal"] = "PAR"
    team["Philadelphia Fusion"] = "PHI"
    team["San Francisco Shock"] = "SFS"
    team["Seoul Dynasty"] = "SEO"
    team["Shanghai Dragons"] = "SHD"
    team["Toronto Defiant"] = "TOR"
    team["Vancouver Titans"] = "VAN"
    team["Washington Justice"] = "WAS"

    mp = dict()

    mp["Ilios"] = "Control"
    mp["Busan"] = "Control"
    mp["Nepal"] = "Control"
    mp["Oasis"] = "Control"
    mp["Lijiang Tower"] = "Control"

    mp["Havana"] = "Escort"
    mp["Rialto"] = "Escort"
    mp["Watchpoint: Gibraltar"] = "Escort"
    mp["Dorado"] = "Escort"
    mp["Junkertown"] = "Escort"
    mp["Route 66"] = "Escort"

    mp["Horizon Lunar Colony"] = "Assault"
    mp["Hanamura"] = "Assault"
    mp["Temple of Anubis"] = "Assault"
    mp["Volskaya Industries"] = "Assault"
    mp["Paris"] = "Assault"

    mp["Eichenwalde"] = "Hybrid"
    mp["King's Row"] = "Hybrid"
    mp["Blizzard World"] = "Hybrid"
    mp["Numbani"] = "Hybrid"
    mp["Hollywood"] = "Hybrid"
    
    mpd = dict()
    ctl = []
    esc = []
    ast = []
    hyb = []
    
    mpd["Ilios"] = ctl
    mpd["Busan"] = ctl
    mpd["Nepal"] = ctl
    mpd["Oasis"] = ctl
    mpd["Lijiang Tower"] = ctl

    mpd["Havana"] = esc
    mpd["Rialto"] = esc
    mpd["Watchpoint: Gibraltar"] = esc
    mpd["Dorado"] = esc
    mpd["Junkertown"] = esc
    mpd["Route 66"] = esc

    mpd["Horizon Lunar Colony"] = ast
    mpd["Hanamura"] = ast
    mpd["Temple of Anubis"] = ast
    mpd["Volskaya Industries"] = ast
    mpd["Paris"] = ast

    mpd["Eichenwalde"] = hyb
    mpd["King's Row"] = hyb
    mpd["Blizzard World"] = hyb
    mpd["Numbani"] = hyb
    mpd["Hollywood"] = hyb
    
#
#
#
#
#
def updateELO(rating, teamA, teamB, sA, sB):
    if (sA > sB):
        sA = 1
    elif( sA < sB):
        sA = 0
    else:
        sA = .5
    
    sB = 1 - sA
    
    qA = 10**(rating[teamA] / 400)
    qB = 10**(rating[teamB] / 400)
    
    eA = qA/(qA + qB)
    eB = qB/(qA + qB)
    
    rating[teamA] = rating[teamA] + 20 * (sA - eA)
    rating[teamB] = rating[teamB] + 20 * (sB - eB)
    
    return rating

In [17]:
df = pd.read_csv("match_map_stats.csv")

dictSetup()

elo_ = pd.read_csv( "ELO.csv").
elo = elo_.iloc[-1,:]
control = pd.read_csv( "ControlELO.csv")
ctl = control.iloc[-1,:]
hybrid = pd.read_csv( "HybridELO.csv")
hyb = hybrid.iloc[-1,:]
assualt = pd.read_csv( "AssaultELO.csv")
ast = assualt.iloc[-1,:]
escort = pd.read_csv( "EscortELO.csv")
esc = escort.iloc[-1,:]

df = df[df["map_round"] == 1]
df = df[['match_id', 'map_winner', 'map_loser', 'map_name',
         'winning_team_final_map_score', 'losing_team_final_map_score',
         'attacker', 'defender']]

cur = df.iloc[0,0]

for mid, mwin, mlos, mapn, wscore, lscore, atk, dfn in df.values:    
    # write to csv if new match
    if cur != mid:
        cur = mid

    if mwin == "draw":
        mwin = atk
        mlos = dfn
    
    #updateELO(mpd[mapn], team[mwin], team[mlos], wscore, lscore)


In [19]:
ctl

ATL    1500
BOS    1500
CHD    1500
DAL    1500
FLA    1500
GLA    1500
GZC    1500
HOU    1500
HZS    1500
LDN    1500
NYE    1500
PAR    1500
PHI    1500
SFS    1500
SEO    1500
SHD    1500
TOR    1500
VAL    1500
VAN    1500
WAS    1500
Name: 0, dtype: int64